In [1]:
from cProfile import label

import numpy as np
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

## Mean-squared error

In [ ]:
# loss function
lossFnMSE = nn.MSELoss()

# create predictions and real answer
yHat = torch.linspace(-2,2,101)
y = torch.tensor(.5)

# compute MSE loss function
L = np.zeros(101)
for i,yy in enumerate(yHat):
    L[i] = lossFnMSE(yy,y)

plt.plot(yHat,L,label="Loss")
plt.plot([y,y],[0,np.max(L)],"r--",label="True value")
plt.ylabel('Predicted value')
plt.legend()
plt.show()